# FYSC12: Beta decay and Fermi Kurie plot
## Data analysis code and lab notebook

### Introduction
* This 'Jupyter Notebook' is your manual, data analysis and log book all at the same time. It consists of individual cells of text or Python code and you can edit or add cells to your heart's content.
* When using the notebook, there are two modes: *command* and *edit* mode. In command mode, you can add, move, run or delete cells. In edit mode, you modify its content. Use the *Esc* key or click outside the cell to leave edit mode. With *shift* + *return* you can run it directly from edit mode.
* When you 'run' a cell it will either be formated or executed depending on whether it is a text cell or a code cell, respectivly. The output will be shown inside the notebook.
* It is best to run the notebook piece-by-piece as we are progressing through the lab. If something doesn't work as expected, try 'Kernel->Restart' to get a fresh working environment.
* Code cells support typical editing features of an IDE, such as tab-complete. You find more info in the 'Help' menu including all keyboard shortcuts!
* **Please note**: The code is not supposed to work out-of-the-box but needs some tweaks from your side to complete all the way through! So if you encounter an error, don't despair, try to fix it and ask for help should you get stuck!

### Some basic things on Python:

In [ ]:
## assign a variable:
myvar = 23.5
## print a variable to the notebook:
print(myvar)
## print text + variable:
print('this is myvar: {}'.format(myvar))
## create an array of three elements:
myothervar = [ 2, 5, 9 ]
print ("Array:")
print (myothervar)
print ("First element is:")
print (myothervar[0])
## help with any function:
help(print)

For data storage and analysis, we rely on the *numpy* and *scipy* packages. And to plot things, we use the *matplotlib* library. We need to import both before the first use and initialize the interactive plots inside the notebook:

In [ ]:
import numpy as np
import scipy
import matplotlib
# choose a backend for web applications; remove for stand-alone applications:
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# enable interactive notebook plots (
# alternative: use 'inline' instead of 'notebook' for static images)
%matplotlib notebook

### Let's take some data!
#### describe your experiment and settings here:

.......

**Important**: write down any settings you apply! And take pictures of the setup/sources.

#### Load the data from the MCA
We have a little helper library called MCA.py that we load here. It provides a new class *Spectrum* and a routine to load (ASCII) files from the MCA: *load_spectrum("filename")*

In [ ]:
import MCA
sr90 = MCA.load_spectrum(filename =  'data/mydatafile.Spe')
sr90.name = "Sr-90"

Now it's time to plot the data we just measured!
Run the cell below to see the result appear inside the notebook:

### Plot the first data

In [ ]:
fig, axes = plt.subplots() # makes new plot
axes.plot(sr90.x, sr90.y, linestyle="steps")

## Could be useful to see this in log scale..?
#plt.yscale('log')
#plt.ylim(ymin=1)

The result looks (hopefully) more-or-less identical to what we have seen in the MCA software. Good, let's proceed with the analysis then!

**Note**: if you use a pure Python program (not a notebook like we do today) and want to plot with matplotlib, then you need to adjust the indicated lines around the matplotlib import statement add the *plt.show()* command (without arguments) to actually _show_ the plots in a separate window.

#### Questions:
- what is the Q value (in channels) that you read off the plot above? How confident (in units of channels) are you with that reading?

**Answer here**:

## Energy calibration of the detector
We have two different sources available: **Cs-137** and **Bi-207**

Your task:
- have your lab supervisor hand you a source and instruct you to place it at the measurement location
- take a spectrum
- while waiting for the measurement: study the incoming MCA spectrum (alternatively, plot the already recorded data here) or research/calculate the energies of the peaks (see below)

When you are ready, plot the new raw data:

In [ ]:
calsource1 = MCA.load_spectrum(filename = ' ')
calsource1.name = "Bi-207"

fig, axes = plt.subplots() # makes new plot
plt.plot(calsource1.x, calsource1.y, linestyle="steps", label = calsource1.name)

## Let's make this plot a little nicer!

#plt.yscale('log')
#plt.xlabel('')
#plt.ylabel('')
#plt.title('')
#plt.legend()


### Identifying the peaks in the spectrum and their associated energies
- research at what intensities and energies the source emitts internal conversion electrons (electron capture process). *Hint:* The *Nudat* database might be useful: http://www.nndc.bnl.gov/nudat2
- identify what emissions contribute to the peaks you see in the spectrum
- our detector cannot resolve small differences in energy, so the peaks we see might stem from several emissions at different energies; calculate the mean energy taking into account the (relative) intensities

In [ ]:
calsource1_Epeak = np.array([ 1, 2, 3, 4 ]) # Energies in MeV

for peak in calsource1_Epeak:
    print ("E peak: {} MeV".format(peak))

### Determining the IC peak position and width via Gaussian fits

- what is the advantage of using Gaussian fits over reading off the value by eye?
- *optional*: let's try to fit even the third (and weakest) peak of Bi-207! Make it visible in the data plot and perform a Gaussian fit at the position you identified by eye using *fithelpers.fit_gaussian_at_pos()*.

In [ ]:
import fithelpers
gfits = fithelpers.fit_all_gaussians(calsource1.x, calsource1.y)

# if the fit did not work, you can add fits at perfomed at specific positions *pos* instead:
# gfits.append(fithelpers.fit_gaussian_at_pos(calsource1.x, calsource1.y, pos=....))

# now you can access the fitted gaussians and their coefficients:
print ("Received {} Gaussians from fit routine.".format(len(gfits)))
for idx, g in enumerate(gfits):
    print("Fitted Gaussian #{}: A: {}, mu: {}, sigma: {}".format(idx, g.A, g.mu, g.sigma))

Now let's plot the data and each of the Gaussian fits!

In [ ]:
## set up a new plot
fig, axes = plt.subplots() # makes new plot
plt.grid(True)
plt.xlabel('channel number')
plt.ylabel('counts')
plt.title(calsource1.name)

## plot the calibration source's data
plt.plot(calsource1.x, calsource1.y, linestyle='steps',label=calsource1.name)

## loop over all the fit results and enumerate them with an index
for idx, g in enumerate(gfits):
    ## print a line with the fit result
    print("Fitted Gaussian #{}: A: {}, mu: {}, sigma: {}".format(idx, g.A, g.mu, g.sigma))
    ## plot this gaussian fit too
    plt.plot(calsource1.x, g.value(calsource1.x), 
             label="Gauss fit {}".format(idx))
    ## add a label to the peak position with the fit index
    plt.annotate(idx,
            xy=(g.mu, g.A), xycoords='data', # where to point to
            xytext=(15, 15), textcoords='offset points', # offset
            arrowprops=dict(arrowstyle="->")) # arrowstyle
    
leg = plt.legend() # make legend visible


### Match the fitted peaks to the correct energies

In [ ]:
# now put the peak positions into a list to plot it later
# HINT: use the coefficients from the fit! E.g. 'gfits[0].mu'
calsource1_channel = np.array([ 1, 2, 3 , 4 ])
calsource1_sigma   = np.array([ 0.1, 0.2, 0.3, 0.4])

# now print the values:
print ("Energies:")
print (calsource1_Epeak)
print ("Channels:")
print (calsource1_channel)
print ("Sigma:")
print (calsource1_sigma)
# or a little more fancy:
#for idx, p in enumerate(calsource1_channel):
#    print ("Peak #{}: E = {} MeV, channel = {} +/- {}"
#           .format(idx, calsource1_Epeak[idx], p, calsource1_sigma[idx]))

### *Optional*: Now repeat the above steps for the 2nd calibration source
- start the data taking and study the incoming MCA spectrum 
- research at what intensities and energies the source emitts internal conversion electrons 
- identify what emissions contribute to the peaks we see
- calculate the mean energy taking into account the (relative) intensities

In [ ]:
calsource2 = MCA.load_spectrum(filename = '')
# only continue if the source was loaded correctly:
if calsource2:
    calsource2.name = "Toulouse"
    gfits2 = fithelpers.fit_all_gaussians(calsource2.x, calsource2.y, loglevel="WARNING")
    
    fig, axes = plt.subplots() # makes new plot
    plt.plot(calsource2.x, calsource2.y, linestyle="steps", label = calsource2.name)

    for idx, g in enumerate(gfits2):
        print("Fitted Gaussian #{}: {}".format(idx, g.as_string()))
        plt.plot(calsource2.x, g.value(calsource2.x), 
                 label="Gauss fit {}".format(idx))
        plt.annotate(idx,
                xy=(g.mu, g.A), xycoords='data', # where to point to
                xytext=(15, 15), textcoords='offset points', # offset
                arrowprops=dict(arrowstyle="->")) # arrowstyle
    
    ## Let's make this plot a little nicer!
    plt.xlabel('channel number')
    plt.ylabel('counts')
    plt.title(calsource2.name)
    leg = plt.legend()

In [ ]:
calsource2_Epeak = np.array([ 1, 2, 3, 4 ]) # Energies in MeV

for peak in calsource2_Epeak:
    print ("E peak: {} MeV".format(peak))

Now assemble above results into data structures we can plot later

In [ ]:
calsource2_channel = np.array([ 1, 2, 3 , 4 ])
calsource2_sigma   = np.array([ 0.1, 0.2, 0.3, 0.4])

### Plot the energy calibration figure

In [ ]:
fig, axes = plt.subplots() # makes new plot
plt.grid(True)
plt.xlabel('channel number')
plt.ylabel('energy [MeV]')
plt.title("Energy Calibration")
## PLOT the energy calibration data including uncertainties
plt.errorbar(x=calsource1_channel, 
             y=calsource1_Epeak, 
             xerr=calsource1_sigma, 
             marker='o',
             label=calsource1.name)

## might want to COMBINE data arrays from different calibration sources for the fit:
## use
## new_list = list1 + list2
## to do so. Then change the data set in the fit command.

## linear regression of the data
## http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
## or "help(scipy.stats.linregress)"
slope = .1
intercept = 0.
# .... something is missing here....

## ADVANCED METHOD TO FIT:
## use "curve_fit" which allows to take uncertainties into account!
## http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
## or "help(scipy.optimize.curve_fit)"

print("Determined calibration \
from linear regression: E [MeV] \
= {}*N_ch + {}".format(slope, intercept))

x = np.arange(1,512)
plt.plot(x,slope*x+intercept,label="linear regression")
plt.legend()

## apply energy calibration
print("Reloading {} spectrum".format(sr90.name))
sr90 = MCA.load_spectrum(filename =  sr90.filename)
print("Applying calibration constants")
sr90.x = sr90.x*slope + intercept

fig, axes = plt.subplots() # makes new plot
plt.grid(True)
plt.ylabel('counts')
plt.xlabel('energy [MeV]')
plt.title("Sr-90 spectrum")
plt.plot(sr90.x,sr90.y,label=sr90.name)


### Energy calibration, optional steps:
- can you demonstrate that the peaks we have identified in fact stem from electrons? How?
- improve the calibration by taking into account all IC emissions from Bi-207 and Cs-137
- improve the fit by taking into account uncertainties!
- how can you evaluate the quality of the fit? Look into the documentation (or code) of the routine and interpret what information it returns!

## Fermi-Kurie Plot
Let's calculate $Q-T_e$ and look at the resulting distribution!

*You are lucky, all the formulars have already been typed in -- and look how easy Python (with numpy) makes the calculations!*

In [ ]:
## fermi-kurie calculations:
mec2 = 0.510998910 ## e mass in MeV
pc = np.sqrt((sr90.x + mec2)**2 - mec2**2)
A = pc/mec2
f = 1.3604*A*A + 0.1973*A + 0.0439
Ee = (sr90.x + mec2)
QminTe = np.sqrt((sr90.y*pc)/(Ee*f))

## not plot the calculated data
fig, axes = plt.subplots() # makes new plot
plt.grid(True)
plt.xlabel('Te [MeV]')
plt.ylabel('Q-Te')
plt.title("{} Fermi-Kurie".format(sr90.name))
plt.plot(sr90.x, QminTe, label="data", linestyle="steps")
    
## linear regression of the F-K plot see
## http://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.linregress.html
## the fit does not really work on the edges of the F-K plot, so we
## take the linear region of lower_limit<E [MeV]<upper_limit (to
## be determined from the plot)
lower_limit, upper_limit = 1,11 ## initialize
try:
    ## search for the bins that match our criteria
    ## first elements ([0][0]) indicate first bin matching our criteria
    lower_limit = np.where(sr90.x>0.1)[0][0]
    upper_limit = np.where(sr90.x>0.5)[0][0]
except IndexError:
    print("Could not find any bins to fit! Please check the limit settings!")

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    sr90.x[lower_limit:upper_limit], QminTe[lower_limit:upper_limit])

x = np.arange(0,2.5,0.05) ## generate x axis for fit result (start, stop, stepsize)
plt.plot(x,slope*x+intercept,label="linear regression")

## now the Q value is determined by where the linear regression 
## intersects with the x axis (Q-Te = 0)
Q = -intercept/slope

## print results
print("Determined linear regression to Fermi-Kurie plot: Q-Te = " 
          + str(slope)+"*Te + " + str(intercept))
print("===> Q value: Q = "+str(Q)+" MeV ")
## label plot with Q value
plt.annotate(r'$Q_{Y-90}$ = '+"{:.3f}".format(Q)+' MeV', # text to put there
            xy=(Q, 0),                         # where to point to
            xytext=(0., 60),                   # offset for text
            textcoords='offset points',
            arrowprops=dict(arrowstyle="->", color = 'red'))


### Discussion of the result: estimating the systematic uncertainty
**Question**: what is the Q value you get from the F-K plot?

*Answer*:

**Question**: how does the Q value change if you adjust the fitting range?

*Answer*:

**Question**: how does the Q value change if you modify the energy calibration (e.g. by adding 1 $\sigma$ to the channel value)?

*Answer*: